In [1]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import torch
fp=glob.glob('output_land_patches/patches_*.nc')
fp=sorted(fp)
land_fractL=[]
tc_L=[]
sfc_type_L=[]
sk_temp_L=[]
sk_temp1_L=[]
xenc_L=[]
xenc_prec_L=[]
oe_wvp_L=[]
near_sfc_precip_L=[]
sfc_emiss_L=[]
cldw_oe_L=[]
sfc_bin_L=[]
qv_L=[]
import pickle

#dense_autoencoder=torch.jit.load('dense_autoencoder.pt')
#with open('scaler_qv.pkl','wb') as f:
#    pickle.dump(scaler_qv,f)
#scripted_model = torch.jit.script(model)
#scripted_model.save('dense_autoencoder.pt')



In [2]:
tc_L=[]
sfc_type_L=[]
sk_temp_L=[]
sk_temp1_L=[]
xenc_L=[]
x_env_enc_L=[]
xenc_prec_L=[]
oe_wvp_L=[]
near_sfc_precip_L=[]
sfc_emiss_L=[]
cldw_oe_L=[]
sfc_bin_L=[]
qv_L=[]
from numba import jit
@jit(nopython=True)
def get_qv_sfc(qv,sfc_bin,qv_sfc):
    n_patches,nscans,nrays,nlevs=qv.shape
    for i in range(n_patches):
        for j in range(nscans):
            for k in range(nrays):
                qv_sfc[i,j,k]=qv[i,j,k,sfc_bin[i,j,k]]
    return qv_sfc
import tqdm
for f in tqdm.tqdm(fp[::]):
    with nc.Dataset(f) as df:
        tc_s1=df['tc_s1'][:]
        tc_s2=df['tc_s2'][:]
        surface_type=df['surface_type'][:]
        sk_temp=df['sk_temp'][:]
        qv=df['qv'][:]
        qv_oe=df['qv_oe'][:]
        cldw_oe=df['cldw_oe'][:]
        oe_wvp=df['oe_wvp'][:]
        near_sfc_precip=df['near_sfc_precip'][:]
        sfc_emiss=df['sfc_emiss'][:]
        sfc_bin=df['sfc_bin'][:]
        x_enc_prec=df['x_enc'][:]
        #print(tc_s1.shape)
        n_patches=tc_s1.shape[0]
        qv_sfc=get_qv_sfc(qv.data,sfc_bin.data,np.zeros((n_patches,tc_s1.shape[1],tc_s1.shape[2]),dtype=np.float32))
        #print(qv_sfc.shape)
        #print(qv_sfc.shape)
        for i in range(n_patches):
            a_nan=np.isnan(near_sfc_precip[i,:,24].data)
            if sum(a_nan) > 0:
                continue
            if near_sfc_precip[i,:,24].data.min() <0:
                continue
            if oe_wvp[i,24].min() <0:
                continue
            if tc_s1[i,:,24,:].min() <0:
                continue
            if tc_s2[i,:,24,:].min() <0:
                continue
            tc_L.append(np.concatenate((tc_s1[i,:,24,:],tc_s2[i,:,24,:]),axis=-1))
            sfc_type_L.append(surface_type[i,:,24])
            sk_temp_L.append(sk_temp[i,:,24])
            qv_L.append(qv_sfc[i,:,24])
        #xenc_L.append(x_enc[i,:,:,:])
        #x_env_enc_L.append(x_env_enc[i,:,:,:])
            xenc_prec_L.append(x_enc_prec[i,:,24,:])
            oe_wvp_L.append(oe_wvp[i,:,24])
            sfc_emiss_L.append(sfc_emiss[i,:,24,:])
            log_precip=np.log10(1+near_sfc_precip[i,:,24].data/0.1)
            a_nan=np.nonzero(log_precip!=log_precip)
            if len(a_nan[0]) > 0:
                stop
            near_sfc_precip_L.append(np.log10(1+near_sfc_precip[i,:,24]/0.1))
        #break
    #break
tc_L=np.concatenate(tc_L,axis=0)
sfc_type_L=np.concatenate(sfc_type_L,axis=0)
sk_temp_L=np.concatenate(sk_temp_L,axis=0)
xenc_prec_L=np.concatenate(xenc_prec_L,axis=0)
oe_wvp_L=np.concatenate(oe_wvp_L,axis=0)
sfc_emiss_L=np.concatenate(sfc_emiss_L,axis=0)
near_sfc_precip_L=np.concatenate(near_sfc_precip_L,axis=0)
qv_L=np.concatenate(qv_L,axis=0)
print(tc_L.shape)

100%|██████████| 385/385 [00:41<00:00,  9.30it/s]


(886392, 13)


In [3]:
print(qv_L.min(),oe_wvp_L.min())
print(qv_L.shape,oe_wvp_L.shape)

print(np.corrcoef(qv_L,oe_wvp_L))
print(sfc_type_L.min(),sfc_type_L.max())
a=np.nonzero(sfc_type_L==0)
print(a[0].shape)
b=np.nonzero(sfc_type_L!=0)
print(b[0].shape)


0.0 0.038790476
(886392,) (886392,)
[[1.         0.80197684]
 [0.80197684 1.        ]]
0 324
(67009,)
(819383,)


In [15]:
print(near_sfc_precip_L.min(),near_sfc_precip_L.max(),near_sfc_precip_L.mean())
print(near_sfc_precip_L.shape)

0.0 3.1356606 0.047525432
(886392,)


In [4]:
import xarray as xr
tcX=xr.DataArray(tc_L,dims=['n_profiles','n_chans'],coords={'n_profiles':np.arange(tc_L.shape[0]),'n_chans':np.arange(tc_L.shape[1])})
sfc_typeX=xr.DataArray(sfc_type_L,dims=['n_profiles'],coords={'n_profiles':np.arange(sfc_type_L.shape[0])})
sk_tempX=xr.DataArray(sk_temp_L,dims=['n_profiles'],coords={'n_profiles':np.arange(sk_temp_L.shape[0])})
qvX=xr.DataArray(qv_L,dims=['n_profiles'],coords={'n_profiles':np.arange(qv_L.shape[0])})
oe_wvpX=xr.DataArray(oe_wvp_L,dims=['n_profiles'],coords={'n_profiles':np.arange(oe_wvp_L.shape[0])})
near_sfc_precipX=xr.DataArray(near_sfc_precip_L,dims=['n_profiles'],coords={'n_profiles':np.arange(near_sfc_precip_L.shape[0])})
x_enc_precX=xr.DataArray(xenc_prec_L,dims=['n_profiles','n_comp'],coords={'n_profiles':np.arange(xenc_prec_L.shape[0]),'n_comp':np.arange(xenc_prec_L.shape[1])})
ds=xr.Dataset({'tc':tcX,'sfc_type':sfc_typeX,'sk_temp':sk_tempX,'qv':qvX,'oe_wvp':oe_wvpX,'near_sfc_precip':near_sfc_precipX,'x_enc_prec':x_enc_precX})
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf('training_1d_data_land.nc',encoding=encoding)

In [5]:
d_scaler={}
for vars in ['tc','sfc_type','sk_temp','qv','oe_wvp','near_sfc_precip','x_enc_prec']:
    #print(vars,ds[vars][:].mean(axis=0),ds[vars][:].std(axis=0))
    d_scaler[vars]=[ds[vars][:].data.mean(axis=0),ds[vars][:].data.std(axis=0)]
with open('scaler_1d_land.pkl','wb') as f:
    pickle.dump(d_scaler,f)

# write the d_scaler to a text file easy to read in Fortran
with open('scaler_1d_land.txt', 'w') as f:
    for key, value in d_scaler.items():
        f.write("%s\n"%key)
        mean_text=''
        try:
            for v in value[0]:
                mean_text+=str(v)+' '
        except:
            mean_text+=str(value[0])+' '
        f.write("%s\n"%mean_text)
        std_text=''
        try:
            for v in value[1]:
                std_text+=str(v)+' '
        except:
            std_text+=str(value[1])+' '
        f.write("%s\n"%std_text)
f.close()
print(d_scaler)

{'tc': [array([262.15393, 242.96167, 263.8994 , 247.69026, 265.37137, 261.23865,
       248.13664, 262.6147 , 255.16196, 263.83707, 261.55557, 255.9773 ,
       262.22192], dtype=float32), array([31.05876 , 46.141956, 27.561977, 41.09737 , 24.578712, 28.132069,
       37.89757 , 28.753544, 32.18047 , 24.599308, 26.546144,  9.781629,
       15.644364], dtype=float32)], 'sfc_type': [111.37076710981147, 44.74047868686988], 'sk_temp': [283.65118, 19.78732], 'qv': [3.9666257, 4.375533], 'oe_wvp': [18.048836, 15.562971], 'near_sfc_precip': [0.047525432, 0.22921856], 'x_enc_prec': [array([-0.03576973, -0.01128846,  0.00253951, -0.04949954,  0.0063798 ,
        0.03153232]), array([0.38198821, 0.30524483, 0.23588714, 0.39220661, 0.4068045 ,
       0.24919357])]}


In [34]:
print(value)
print(key)

[111.37076710981147, 44.74047868686988]
sfc_type


In [6]:
tc_scaled=(tcX.data-d_scaler['tc'][0])/d_scaler['tc'][1]
sfc_type_scaled=(sfc_typeX.data-d_scaler['sfc_type'][0])/d_scaler['sfc_type'][1]
sk_temp_scaled=(sk_tempX.data-d_scaler['sk_temp'][0])/d_scaler['sk_temp'][1]
qv_scaled=(qvX.data-d_scaler['qv'][0])/d_scaler['qv'][1]
oe_wvp_scaled=(oe_wvpX.data-d_scaler['oe_wvp'][0])/d_scaler['oe_wvp'][1]
near_sfc_precip_scaled=(near_sfc_precipX.data-d_scaler['near_sfc_precip'][0])/d_scaler['near_sfc_precip'][1]
x_enc_prec_scaled=(x_enc_precX.data-d_scaler['x_enc_prec'][0])/d_scaler['x_enc_prec'][1]
print(x_enc_prec_scaled.shape)
print(near_sfc_precip_scaled.shape)
print(near_sfc_precip_L.shape)

X_input=torch.tensor(np.concatenate((tc_scaled,sfc_type_scaled[:,np.newaxis],sk_temp_scaled[:,np.newaxis],qv_scaled[:,np.newaxis]),axis=-1),dtype=torch.float32)
y_output=torch.tensor(np.concatenate((oe_wvp_scaled[:,np.newaxis],near_sfc_precip_scaled[:,np.newaxis],x_enc_prec_scaled),axis=-1),dtype=torch.float32)

(886392, 6)
(886392,)
(886392,)


In [7]:
ntot=X_input.shape[0]
n_train=int(0.8*ntot)
import torch
X_train=torch.utils.data.TensorDataset(X_input[:n_train],y_output[:n_train])
X_test=torch.utils.data.TensorDataset(X_input[n_train:],y_output[n_train:])
X_train_loader = torch.utils.data.DataLoader(X_train, batch_size=64, shuffle=True)


In [8]:
class DenseModel(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(DenseModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 128)
        self.dropout1 = torch.nn.Dropout(p=0.05)
        self.fc2 = torch.nn.Linear(128, 64)
        self.dropout2 = torch.nn.Dropout(p=0.05)
        self.fc3 = torch.nn.Linear(64, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        #x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        #x = self.dropout2(x)
        x = self.fc3(x)
        return x
input_size = X_input.shape[1]
output_size = y_output.shape[1]
model = DenseModel(input_size, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()
num_epochs = 50
losses = []


In [9]:
num_epochs = 20
for epoch in range(num_epochs):
    avg_loss = 0
    icount=0
    for i, (inputs, targets) in enumerate(X_train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        avg_loss+=loss.item()
        icount+=1
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss/icount:.4f}')
torch.save(model.state_dict(), 'dense_model_land.pth')


Epoch [1/20], Loss: 0.4874
Epoch [2/20], Loss: 0.4406
Epoch [3/20], Loss: 0.4279
Epoch [4/20], Loss: 0.4127
Epoch [5/20], Loss: 0.4066
Epoch [6/20], Loss: 0.3990
Epoch [7/20], Loss: 0.3944
Epoch [8/20], Loss: 0.3908
Epoch [9/20], Loss: 0.3904
Epoch [10/20], Loss: 0.3852
Epoch [11/20], Loss: 0.3817
Epoch [12/20], Loss: 0.3835
Epoch [13/20], Loss: 0.3764
Epoch [14/20], Loss: 0.3783
Epoch [15/20], Loss: 0.3752
Epoch [16/20], Loss: 0.3703
Epoch [17/20], Loss: 0.3727
Epoch [18/20], Loss: 0.3694
Epoch [19/20], Loss: 0.3698
Epoch [20/20], Loss: 0.3630


In [12]:
model.eval()
torch.onnx.export(model, X_input[:1], 'xdense/xdense_model_land.onnx', export_params=True, verbose=True, input_names=['input'], output_names=['output'])

Exported graph: graph(%input : Float(1, 16, strides=[16, 1], requires_grad=0, device=cpu),
      %fc1.weight : Float(128, 16, strides=[16, 1], requires_grad=1, device=cpu),
      %fc1.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %fc2.weight : Float(64, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %fc2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %fc3.weight : Float(8, 64, strides=[64, 1], requires_grad=1, device=cpu),
      %fc3.bias : Float(8, strides=[1], requires_grad=1, device=cpu)):
  %/fc1/Gemm_output_0 : Float(1, 128, strides=[128, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/fc1/Gemm"](%input, %fc1.weight, %fc1.bias), scope: __main__.DenseModel::/torch.nn.modules.linear.Linear::fc1 # /Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/nn/modules/linear.py:125:0
  %/Relu_output_0 : Float(1, 128, strides=[128, 1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="/Relu"](

ltL 28 Number of low resolution 5o grid intervals of latitude from 70oS to
70oN.
lnL 72 Number of low resolution 5o grid intervals of longitude from 180oW
to 180oE.
ltH 536 Number of high resolution 0.25o grid intervals of latitude from 67oS
to 67oN.
lnH 1440 Number of high resolution 0.25o grid intervals of longitude from
180oW to 180oE.
ns 2 Number of swaths:, 0 = Ku+TMI (full swath), 1 = Ku+TMI (narrow swath).
hgt 16 Number of level heights 0-15: 0: near surface, 1-10: height = 1.0km
* index, 11-15: height = 10.0km + 2.0km * (index-10),
tim 24 Number of hourly local time bins.
rt 3 Number of rain types: stratiform, convective, all.
st 3 Number of surface types: ocean, land, all.
bin 30 Number of bins in histogram.
emiss 9 Number of radiometer channel emissivities

In [13]:
y_pred=model(X_input[n_train:]).detach().numpy()
y_test=y_output[n_train:].numpy()
for i in range(6):
    print(np.corrcoef(y_pred[:,i],y_test[:,i]))

[[1.         0.94885651]
 [0.94885651 1.        ]]
[[1.         0.79672945]
 [0.79672945 1.        ]]
[[1.         0.80390849]
 [0.80390849 1.        ]]
[[1.         0.73388527]
 [0.73388527 1.        ]]
[[1.         0.66082102]
 [0.66082102 1.        ]]
[[1.         0.81777793]
 [0.81777793 1.        ]]
